In [2]:
import numpy as np
import pandas as pd

In [22]:
#I want to create course weekly scheduler for my courses.
#Create a dataframe with 9 rows and 6 columns.
rows=['1', '2', '3', '4', 'L', '5', '6', '7', '8']
cols=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
df = pd.DataFrame(index=rows, columns=cols, data='-')

In [ ]:
''' Course1 EEE 313-1	Electronic Circuit Design	
Erdinç Tatar	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	4	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 15:30-17:20
Tue 08:30-12:20
Thu 10:30-12:20 '''
Course1 = ['EEE-313', '01', 'Mon', '15:30-17:20', 'Tue', '08:30-12:20', 'Thu', '10:30-12:20', ]

''' Course2  EEE 313-2	Electronic Circuit Design	
Erdinç Tatar	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	4	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 13:30-15:20
Tue 13:30-17:20
Thu 08:30-10:20 '''
Course2 = ['EEE-313', '02', 'Mon', '13:30-15:20', 'Tue', '13:30-17:20', 'Thu', '08:30-10:20']
''' Course3 '''

''' Course4 '''

''' Course5 '''


In [17]:
import re

class CourseScheduleExtractor:
    def __init__(self, schedule_string):
        self.schedule_string = schedule_string

    def extract_course_name(self):
        pattern = r'\b[A-Z]{3} \d{3}-\d+\s+(.+)\s+'
        match = re.search(pattern, self.schedule_string)
        if match:
            return match.group(1).strip()
        return None

    def extract_course_code(self):
        pattern = r'\b([A-Z]{3} \d{3})-\d'
        match = re.search(pattern, self.schedule_string)
        if match:
            return match.group(1).strip().split(' ')[0] + '-' + match.group(1).strip().split(' ')[1]
        return None

    def extract_section_number(self):
        pattern = r'\b[A-Z]{3} \d{3}-(\d)'
        match = re.search(pattern, self.schedule_string)
        if match:
            return match.group(1).strip()
        return None

    def extract_weekdays(self):
        pattern = r'([A-Za-z]{3}) \d{2}:\d{2}-\d{2}:\d{2}'
        matches = re.findall(pattern, self.schedule_string)
        return matches

    def extract_hours(self):
        pattern = r'[A-Za-z]{3} (\d{2}:\d{2}-\d{2}:\d{2})'
        matches = re.findall(pattern, self.schedule_string)
        return matches

    def extract_instructor_name(self):
        pattern = r'\n(\b[A-Za-zÇçĞğİıÖöŞşÜü\s]+)\s+header'
        match = re.search(pattern, self.schedule_string)
        if match:
            return match.group(1).strip()
        return None




In [18]:
# Example usage:
schedule_string = '''EEE 313-2    Electronic Circuit Design    
Erdinç Tatar    header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3     4     4     6.5    55 Mand. or Elect.    55    0    0    0    55 Mand. or Elect.    55    Mon 13:30-15:20
Tue 13:30-17:20
Thu 08:30-10:20
header=[Hint] body=[Syllabus]  '''

extractor = CourseScheduleExtractor(schedule_string)

course_name = extractor.extract_course_name()
course_code = extractor.extract_course_code()
section_number = extractor.extract_section_number()
weekdays = extractor.extract_weekdays()
hours = extractor.extract_hours()
instructor_name = extractor.extract_instructor_name()

print("Course Name:", course_name)
print("Course Code:", course_code)
print("Section Number:", section_number)
print("Weekdays:", weekdays)
print("Hours:", hours)
print("Instructor Name:", instructor_name)

Course Name: Electronic Circuit Design
Course Code: EEE-313
Section Number: 2
Weekdays: ['Mon', 'Tue', 'Thu']
Hours: ['13:30-15:20', '13:30-17:20', '08:30-10:20']
Instructor Name: Erdinç Tatar


In [21]:
# Example usage:
schedule_string = '''EEE 313-1	Electronic Circuit Design	
Erdinç Tatar	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	4	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 15:30-17:20
Tue 08:30-12:20
Thu 10:30-12:20
header=[Hint] body=[Syllabus]  
EEE 313-2	Electronic Circuit Design	
Erdinç Tatar	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	4	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 13:30-15:20
Tue 13:30-17:20
Thu 08:30-10:20
header=[Hint] body=[Syllabus]  
EEE 313-3	Electronic Circuit Design	
Erdinç Tatar	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	4	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Tue 10:30-12:20
Thu 15:30-17:20
Fri 08:30-12:20
header=[Hint] body=[Syllabus]    
'''

extractor = CourseScheduleExtractor(schedule_string)

course_name = extractor.extract_course_name()
course_code = extractor.extract_course_code()
section_number = extractor.extract_section_number()
weekdays = extractor.extract_weekdays()
hours = extractor.extract_hours()
instructor_name = extractor.extract_instructor_name()

print("Course Name:", course_name)
print("Course Code:", course_code)
print("Section Number:", section_number)
print("Weekdays:", weekdays)
print("Hours:", hours)
print("Instructor Name:", instructor_name)

Course Name: Signals and Systems
Course Code: EEE-321
Section Number: 1
Weekdays: ['Mon', 'Wed', 'Fri']
Hours: ['08:30-10:20', '13:30-15:20', '15:30-17:20']
Instructor Name: Süleyman Serdar Kozat


In [50]:
import re

class CourseScheduleExtractor:
    def __init__(self, schedule_string):
        self.schedule_string = schedule_string

    def extract_course_info(self):
        pattern = r'(\b[A-Z]{3} \d{3}-\d+)\s+(.+)\s+([A-Za-zÇçĞğİıÖöŞşÜü\s]+)\s+header'
        matches = re.findall(pattern, self.schedule_string)
        course_info_list = []
        new_str= schedule_string.split('header=[Hint] body=[Syllabus]')
        #print(new_str)
        for match in matches:
            weekdays = []
            hours = []
            i = matches.index(match)
            #print(i)
            course_code = match[0].strip()
            course_name = match[1].strip()
            instructor_name = match[2].strip()
            parts = re.findall(r'(Mon|Tue|Wed|Thu|Fri) (\d{2}:\d{2}-\d{2}:\d{2})', new_str[i])
            for part in parts:
                weekdays.append(part[0])
                hours.append(part[1])
            course_info = {
                "Course Code": course_code,
                "Course Name": course_name,
                "Instructor Name": instructor_name,
                "Weekdays": weekdays,
                "Hours": hours
            }
            course_info_list.append(course_info)
        return course_info_list


In [51]:
# Example usage:
schedule_string = '''EEE 313-1    Electronic Circuit Design    
Erdinç Tatar    header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3     4     4     6.5    55 Mand. or Elect.    55    0    0    0    55 Mand. or Elect.    55    Mon 15:30-17:20
Tue 08:30-12:20
Thu 10:30-12:20
header=[Hint] body=[Syllabus]  
EEE 313-2    Electronic Circuit Design    
Erdinç Tatar    header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3     4     4     6.5    55 Mand. or Elect.    55    0    0    0    55 Mand. or Elect.    55    Mon 13:30-15:20
Tue 13:30-17:20
Thu 08:30-10:20
header=[Hint] body=[Syllabus]  
EEE 313-3    Electronic Circuit Design    
Erdinç Tatar    header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3     4     4     6.5    55 Mand. or Elect.    55    0    0    0    55 Mand. or Elect.    55    Tue 10:30-12:20
Thu 15:30-17:20
Fri 08:30-12:20
header=[Hint] body=[Syllabus]    
EEE 321-1	Signals and Systems	
Süleyman Serdar Kozat	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	2	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 08:30-10:20
Wed 13:30-15:20
Fri 15:30-17:20
header=[Hint] body=[Syllabus]  
EEE 321-2	Signals and Systems	
Süleyman Serdar Kozat	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	2	4	6.5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Mon 10:30-12:20
Wed 15:30-17:20
Thu 13:30-15:20
header=[Hint] body=[Syllabus]  
EEE 351-1	Engineering Electromagnetics	
Vakur Behçet Ertürk	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	65 Mand. or Elect.	65	0	0	0	65 Mand. or Elect.	65	Mon 13:30-15:20
Thu 08:30-10:20
header=[Hint] body=[Syllabus]  
EEE 351-2	Engineering Electromagnetics	
Ayhan Altıntaş	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	65 Mand. or Elect.	65	0	0	0	65 Mand. or Elect.	65	Wed 10:30-12:20
Fri 15:30-17:20
header=[Hint] body=[Syllabus]  
EEE 351-3	Engineering Electromagnetics	
Ayhan Altıntaş	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	65 Mand. or Elect.	65	0	0	0	65 Mand. or Elect.	65	Wed 08:30-10:20
Fri 13:30-15:20
header=[Hint] body=[Syllabus]  
EEE 447-1	Introduction to Robotics	
Billur Barshan	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	40 Mand. or Elect.	40	0	0	0	40 Mand. or Elect.	40	Mon 13:30-15:20
Thu 08:30-10:20
header=[Hint] body=[Syllabus]  
EEE 448-1	Reinforcement Learning and Dynamic Programming	
Muhammed Ömer Sayın	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	50 Mand. or Elect.	50	0	0	0	50 Mand. or Elect.	50	Mon 08:30-10:20
Wed 13:30-15:20
header=[Hint] body=[Syllabus]  
EEE 485-1	Statistical Learning and Data Analytics	
Cem Tekin	header=[Hint] body=[Info & Courses] header=[Hint] body=[Old Courses of Instructor] header=[Hint] body=[Instructor Evaluation]
3	0	3	5	55 Mand. or Elect.	55	0	0	0	55 Mand. or Elect.	55	Tue 08:30-10:20
Thu 13:30-15:20
header=[Hint] body=[Syllabus]  
'''

extractor = CourseScheduleExtractor(schedule_string)
course_info_list = extractor.extract_course_info()

for course_info in course_info_list:
    print("Course Code:", course_info["Course Code"])
    print("Course Name:", course_info["Course Name"])
    print("Instructor Name:", course_info["Instructor Name"])
    print("Weekdays:", course_info["Weekdays"])
    print("Hours:", course_info["Hours"])
    print()

Course Code: EEE 313-1
Course Name: Electronic Circuit Design
Instructor Name: Erdinç Tatar
Weekdays: ['Mon', 'Tue', 'Thu']
Hours: ['15:30-17:20', '08:30-12:20', '10:30-12:20']

Course Code: EEE 313-2
Course Name: Electronic Circuit Design
Instructor Name: Erdinç Tatar
Weekdays: ['Mon', 'Tue', 'Thu']
Hours: ['13:30-15:20', '13:30-17:20', '08:30-10:20']

Course Code: EEE 313-3
Course Name: Electronic Circuit Design
Instructor Name: Erdinç Tatar
Weekdays: ['Tue', 'Thu', 'Fri']
Hours: ['10:30-12:20', '15:30-17:20', '08:30-12:20']

Course Code: EEE 321-1
Course Name: Signals and Systems
Instructor Name: Süleyman Serdar Kozat
Weekdays: ['Mon', 'Wed', 'Fri']
Hours: ['08:30-10:20', '13:30-15:20', '15:30-17:20']

Course Code: EEE 321-2
Course Name: Signals and Systems
Instructor Name: Süleyman Serdar Kozat
Weekdays: ['Mon', 'Wed', 'Thu']
Hours: ['10:30-12:20', '15:30-17:20', '13:30-15:20']

Course Code: EEE 351-1
Course Name: Engineering Electromagnetics
Instructor Name: Vakur Behçet Ertürk
We